In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
dataset = pd.read_csv('RestaurantGeocoded.csv')
dataset.head()

,Locality,RestaurantName,Address,Category,CostForTwo,Cuisines,Ratings,votes,Latitude,Longitude
0,C Scheme,Kanha,"E 62, Bhagat Singh Marg, C Scheme, Jaipur","['Sweet Shop', 'Quick Bites']",₹500,"['North Indian', 'South Indian', 'Fast Food', ...",4.2,995,26.912258,75.800590
1,C Scheme,Stepout Cafe,"P 14, Sehdev Marg, Ashok Nagar, C Scheme, Jaipur","['Café', 'Casual Dining']",₹800,"['Cafe', 'Italian', 'Lebanese', 'Mexican', 'Co...",4.5,1385,26.905899,75.802078
2,C Scheme,Burger Farm,"3 & 4, Trimurty, V-Jai City Point Building, Ah...",['Quick Bites'],₹400,"['Burger', 'Fast Food', 'Beverages']",4.1,1427,26.914246,75.804930
3,C Scheme,Domino's Pizza,"C 18, Silver Square Mall, Bhagwandas Marg, C S...",['Quick Bites'],₹700,"['Pizza', 'Fast Food']",3.3,1258,26.915043,75.809926
4,C Scheme,Thali and More,"C-46 B, Sarojini Marg, Above Punjab National B...",['Casual Dining'],"₹1,100","['North Indian', 'South Indian', 'Chinese', 'F...",4.4,2339,26.910926,75.805679


In [83]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4353 entries, 0 to 4353
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Locality        4353 non-null   object 
 1   RestaurantName  4353 non-null   object 
 2   Address         4353 non-null   object 
 3   Category        4353 non-null   object 
 4   CostForTwo      4353 non-null   int64  
 5   Cuisines        4353 non-null   object 
 6   Ratings         4341 non-null   object 
 7   votes           4353 non-null   object 
 8   Latitude        4278 non-null   float64
 9   Longitude       4278 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 374.1+ KB


We need to change the data type for CostForTwo, Ratings, votes from object to int, float, int respectively.  
Also, Category and Cuisines are in list format, we need to get the strings out of them.

First, let's check different values of cost.

In [73]:
dataset['CostForTwo'].value_counts()

₹300              734
₹200              532
₹400              512
₹500              415
₹250              345
₹150              296
₹350              251
₹600              220
₹100              167
₹450              152
₹700              113
₹800              110
₹1,000             75
₹550               67
₹1,200             51
₹900               43
₹650               42
₹750               32
₹1,500             27
₹1,100             26
₹1,400             20
₹1,600             16
₹850               15
₹2,000             13
₹1,300             13
₹1,800             11
₹3,000              9
₹1,700              7
₹2,500              6
₹950                4
₹2,400              3
₹4,000              3
₹2,100              3
₹3,500              3
₹2,200              3
₹5,000              2
₹2,600              2
₹6,000              2
₹7,000              1
₹2,300              1
₹1,150              1
₹3,200              1
₹1,250              1
₹2,800              1
No Cost given.      1
₹4,100    

No cost given is not a cost, we need to get rid of it.

In [75]:
dataset.loc[dataset['CostForTwo']=='No Cost given.']

,Locality,RestaurantName,Address,Category,CostForTwo,Cuisines,Ratings,votes,Latitude,Longitude
2231,Mansarovar,Burgs & Fries,"Shop 8, Ridhi Sidhi Link Road, Gopal Pura, Man...",[],No Cost given.,"['Fast Food', 'Burger', 'Wraps']",-,No-Votes,26.83875,75.764087


It seems we don't have a category, cost, rating, or votes for this restaurant. With the current amount of data, this tuple is no use to us.

In [77]:
dataset.drop(2231, inplace=True)

With that out of the way, there is no other anomaly in the costs, let's remove the Rupee symbol and comma from the cost and typecast the values to integer.  
We will use regular expression to accomplish that.

In [80]:
cost = dataset['CostForTwo'].values
cost = [re.sub(',','',item) for item in cost]
cost = [item[1:] for item in cost]
cost = [int(item) for item in cost]
type(cost[0])

int

In [81]:
dataset['CostForTwo'] = cost

In [82]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4353 entries, 0 to 4353
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Locality        4353 non-null   object 
 1   RestaurantName  4353 non-null   object 
 2   Address         4353 non-null   object 
 3   Category        4353 non-null   object 
 4   CostForTwo      4353 non-null   int64  
 5   Cuisines        4353 non-null   object 
 6   Ratings         4341 non-null   object 
 7   votes           4353 non-null   object 
 8   Latitude        4278 non-null   float64
 9   Longitude       4278 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 374.1+ KB
